# Fetch a Playlist from Spotify and create a shuffled copy
Prerequisites:
``` 
pip install pyyaml
pip install spotipy
```

Register new app on https://developer.spotify.com/dashboard

Make sure the redirect_uri in the spotify app config is set to the same value as in config.yaml.

Copy client_id and client_secret to config.yaml file.

In [ ]:
import spotipy
import yaml
from spotipy.oauth2 import SpotifyOAuth
from datetime import datetime
import random

Define the name of the an existing user playlist we want to clone:

In [ ]:
playlist_name = "mp BBQ"

## Load config.yaml and authenticate

In [ ]:
with open("config.yaml", "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=config["client_id"],
                                               client_secret=config["client_secret"],
                                               redirect_uri=config["redirect_uri"],
                                               scope=["user-library-read",
                                                      "playlist-read-private",
                                                      "playlist-read-collaborative",
                                                      "playlist-modify-public",
                                                     "playlist-modify-private"]))

## Fetch playlist

In [ ]:
playlists = sp.current_user_playlists()
playlist = None
for item in playlists["items"]:
    if playlist_name == item["name"]:
        playlist = item
        break
        
if playlist is not None:
    print(f'Found playlist: {item["id"]} {item["name"]}');
else:
    print(f'Playlist {playlist_name} not found in {[item["name"] for item in playlists["items"]]}')

## Get trackIds from playlist and shuffle

In [ ]:
track_ids = [entry["track"]["id"] for entry in sp.playlist_items(playlist["id"], fields="items(track(id))")["items"]]
# inline shuffle
random.shuffle(track_ids)

## Create new playlist

In [ ]:
datetime_str = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
suffix = f"_shuffled_{datetime_str}"
shuffled_playlist = sp.user_playlist_create(sp.current_user()["id"], f'{playlist["name"]}{suffix}', public=False, collaborative=False, description=f'Shuffled Version of {playlist["name"]}, created {datetime_str}')

# Add shuffled tracks to new playlist

In [ ]:
sp.user_playlist_add_tracks(sp.current_user()["id"], shuffled_playlist["id"], track_ids)